In [5]:
%load_ext autoreload
%autoreload 2

import os

%matplotlib inline
from pylab import *
import matplotlib.pyplot as plt
import itertools
import random

In [6]:
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb
import pyAgrum.causal as csl
import pyAgrum.causal.notebook as cslnb

In [7]:
bn=gum.BayesNet('Pandemic Deaths')

In [8]:
age_groups = [1, 40]
age_data = {
    "China": {1: [0.005, 24], 40: [0.038, 45]},
    "USA": {1: [0.005, 1], 40: [0.038, 2]},
}
GDPpc_data = {
    "China": 1,
    "USA": 6,
}
countries = ["USA", "China"]
country_emergence = [30, 5]
collapse_val = 7/80
r_data = [2, 1.15, 0.677, 0.3962, 0.22, 0.129, 0.076, 0.047]
v_val = 150

In [9]:
def array_one_prob(index, length, min_val):
    ar = list(repeat('0', length))
    ar[index - min_val] = 1
    return ar

In [10]:
params = {
    "NxRange": [0, 15],
    "SxRange": [0, 5],
    "GDPpcRange": [min(GDPpc_data.values()), max(GDPpc_data.values())],
    'rRange': [0, len(r_data) - 1],
    'vRange': [v_val, v_val]
}
params["DxRange"] = [min(params["NxRange"]) * min(params["SxRange"]), max(params["NxRange"]) * max(params["SxRange"])]
params["D_all_ages_range"] = [min(params["DxRange"]) * len(age_groups), max(params["DxRange"]) * len(age_groups)]
params["deathsRange"] = [min(params["D_all_ages_range"]) * len(r_data), max(params["D_all_ages_range"]) * len(r_data)]
params["global_monetary_lossRange"] = [min(params["deathsRange"]) * v_val * min(params["GDPpcRange"]), max(params["deathsRange"]) * v_val * max(params["GDPpcRange"])]

In [ ]:

bn.add(gum.LabelizedVariable('country', 'where does the pandemic emerge', countries))

nampl=bn.addSUM(gum.RangeVariable("D_all_ages", "Sum of basic death counts per age", params["D_all_ages_range"][0],params["D_all_ages_range"][1]))

for group in age_groups:
    bn.add(gum.RangeVariable('Sx_' + str(group), 'Excess death rate for age group', params["SxRange"][0], params["SxRange"][1]))
    bn.add(gum.RangeVariable('Nx_' + str(group), 'Population of age group in country', params["NxRange"][0], params["NxRange"][1]))
    bn.add(gum.RangeVariable('Dx_' + str(group), 'Deaths for age group', params["DxRange"][0], params["DxRange"][1]))

    bn.addArc(bn.idFromName("country"),bn.idFromName('Nx_' + str(group)))
    bn.addArc(bn.idFromName('Sx_' + str(group)), bn.idFromName('Dx_' + str(group)))
    bn.addArc(bn.idFromName('Nx_' + str(group)), bn.idFromName('Dx_' + str(group)))
    bn.addArc(bn.idFromName('Dx_' + str(group)), bn.idFromName('D_all_ages'))

bn.add(gum.RangeVariable('r', 'probability of pandemic emergence', params["rRange"][0], params["rRange"][1]))
bn.add(gum.RangeVariable('v', 'value of lost years', params["vRange"][0], params["vRange"][1]))
bn.add(gum.RangeVariable('GDP_pc', 'GDP per capita', params["GDPpcRange"][0], params["GDPpcRange"][1]))
bn.add(gum.RangeVariable('deaths', 'total deaths', params["deathsRange"][0], params["deathsRange"][1]))
bn.add(gum.RangeVariable('global_monetary_loss', 'global loss from deaths', params["global_monetary_lossRange"][0], params["global_monetary_lossRange"][1]))

bn.addArc(bn.idFromName('D_all_ages'),bn.idFromName('deaths'))
bn.addArc(bn.idFromName('r'),bn.idFromName('deaths'))
bn.addArc(bn.idFromName('v'),bn.idFromName('global_monetary_loss'))
bn.addArc(bn.idFromName('country'),bn.idFromName('GDP_pc'))
bn.addArc(bn.idFromName('GDP_pc'),bn.idFromName('global_monetary_loss'))
bn.addArc(bn.idFromName('deaths'),bn.idFromName('global_monetary_loss'))

bn.add(gum.RangeVariable('collapse','Causes collapse of civilization?',0, 1))

bn.addArc(bn.idFromName('deaths'),bn.idFromName('collapse'))

In [ ]:
for country in age_data.keys():
    bn.cpt('GDP_pc')[{"country":country}]=array_one_prob(GDPpc_data[country], params["GDPpcRange"][1] - params["GDPpcRange"][0] + 1, params["GDPpcRange"][0])
    for group in age_groups:
        bn.cpt('Sx_' + str(group))[{"country":country}]=array_one_prob(age_data[country][group][0], params["SxRange"][1] - params["SxRange"][0] + 1, params["SxRange"][0])
        bn.cpt('Nx_' + str(group))[{"country":country}]=array_one_prob(age_data[country][group][1], params["NxRange"][1] - params["NxRange"][0] + 1, params["NxRange"][0])

bn.cpt('country').fillWith(country_emergence).normalize()
bn.cpt('r').fillWith(r_data).normalize()
bn.cpt('v')[:] = array_one_prob(v_val, params["vRange"][1] - params["vRange"][0] + 1, params["vRange"][0])

In [ ]:
for group in age_groups:
    bn.cpt('Dx_' + str(group)).fillWithFunction('Sx_' + str(group) + "*" 'Nx_' + str(group))
bn.cpt("deaths").fillWithFunction("r*D_all_ages")
bn.cpt("global_monetary_loss").fillWithFunction("deaths*v*GDP_pc")
bn.cpt('collapse').fillWithFunction("round(deaths/"+str(collapse_val*2)+")")

In [ ]:
bn

In [ ]:
gnb.showInformation(bn,{},size="20")

In [ ]:
ie=gum.LazyPropagation(bn)
ie.setEvidence({})
ie.makeInference()
gnb.showProba(ie.posterior(bn.idFromName("collapse")))

In [76]:
gnb.showInference(bn,evs={})

InvocationException: Program terminated with status: 1. stderr follows: Error: Edge length 140289.000000 larger than maximum 65535 allowed.
Check for overwide node(s).


In [ ]:
gnb.sideBySide(bn.cpt("r0"), bn.cpt("t"), bn.cpt("theta"))

In [ ]:
cm=csl.CausalModel(bn)

cslnb.showCausalImpact(cm,"collapse", "country")